In [3]:
# Install necessary libraries
!pip install -q -U torch transformers accelerate bitsandbytes

مرحبا

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "Qwen/Qwen3-0.6B"

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# Load Model in Full Precision (Float16)
# This requires approx 4GB VRAM, leaving ~10GB+ for context (long conversations)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Full half-precision (no 4-bit/8-bit compression)
    device_map="auto",
    trust_remote_code=True
)

print(f"Full model loaded on {model.device} | VRAM used: {model.get_memory_footprint() / 1024**3:.2f} GB")

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Full model loaded on cuda:0 | VRAM used: 1.11 GB


In [ ]:
import torch
from transformers import TextStreamer, AutoTokenizer, AutoModelForCausalLM

# Ensure model and tokenizer are loaded globally first
# (If you already ran the loading cell, you don't need to reload them,
# but make sure 'tokenizer' and 'model' variables exist)

def chat_with_qwen():
    print("Qwen3-1.7B (Full Precision) Ready! (THINKING MODE LOCKED ON)")
    print("Type 'exit' to stop.\n")

    history = []

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ["exit", "quit"]:
            break

        # Update history
        history.append({"role": "user", "content": user_input})

        # Apply Template with FORCED Thinking
        text = tokenizer.apply_chat_template(
            history,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=True  # <--- HARDCODED TO TRUE
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

        # Initialize the streamer
        streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

        print("Qwen (Thinking...): ", end="")

        # Generate
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=4096,
            temperature=0.6,
            top_p=0.95,
            do_sample=True,
            streamer=streamer
        )

        # Decode for history
        new_tokens = generated_ids[0][len(model_inputs.input_ids[0]):]
        response_text = tokenizer.decode(new_tokens, skip_special_tokens=True)

        history.append({"role": "assistant", "content": response_text})

# Run the chat
chat_with_qwen()

Qwen3-1.7B (Full Precision) Ready! (THINKING MODE LOCKED ON)
Type 'exit' to stop.

Qwen (Thinking...): <think>
Okay, the user just said "مرحبا" which translates to "Hello" in Arabic. I need to respond in Arabic as well. Let me make sure I understand the request correctly. The user might be testing if I can understand Arabic, so I should respond in the same language. I should keep it friendly and welcoming. Let me check the response again to ensure it's correct and matches the user's intent.
</think>

مرحبا! 😊
Qwen (Thinking...): <think>
Okay, the user is asking about their remaining food after having three cakes and one salad. Let me start by understanding their question. They mentioned three cakes and one salad, so they probably have those. Now, they want to know how much they have left. 

First, I need to check if they are asking about the total amount or if there's something else. The phrase "معي ثلاث تفاحات واكلت اثنتين" translates to "I have three cakes and one salad." So, the tot